In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.keras.models import load_model
import random

In [ ]:
nltk.download('punkt')#Sentence tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
keywords = []
classes_list = []
documents_list = []
ignore = ['?', '!']
file_content = open('intents.json').read()
intents_data = json.loads(file_content)

In [ ]:
for intent_data in intents_data['intents']:
    patterns = intent_data['patterns']
    tag = intent_data['tag']

    for pattern in patterns:
        word_tokens = nltk.word_tokenize(pattern)
        keywords.extend(word_tokens)
        documents_list.append((word_tokens, tag))

    if tag not in classes_list:
        classes_list.append(tag)

keywords = [lemmatizer.lemmatize(word.lower()) for word in keywords if word not in ignore]

In [ ]:
keywords = sorted(list(set(keywords)))
classes_list = sorted(list(set(classes_list)))

In [ ]:
print("Keywords = ",len(keywords))
print("List of Class = ",len(classes_list))
print("List of Documents = ", len(documents_list))

Keywords =  421
List of Class =  38
List of Documents =  313


In [ ]:
pickle.dump(keywords,open('words.pkl','wb'))
pickle.dump(classes_list,open('classes.pkl','wb'))

In [ ]:
data_training = []
empty_output = [0] * len(classes_list)

for documents in documents_list:
  bag_of_words = []
  patterns_of_words = documents [0]
  patterns_of_words = [lemmatizer.lemmatize(word.lower()) for word in patterns_of_words]

  for word in keywords:
      if word in patterns_of_words:
          bag_of_words.append(1)
      else:
          bag_of_words.append(0)

  row_output = list(empty_output)
  row_output[classes_list.index(documents[1])] = 1

  data_training.append([bag_of_words, row_output])

In [ ]:
random.shuffle(data_training)

In [ ]:
training_bags = np.array([item[0] for item in data_training])
train_features = np.array([item[1] for item in data_training])

training_data = np.hstack((training_bags, train_features))

In [ ]:
from sklearn.model_selection import train_test_split

train_input = [data[:len(keywords)] for data in training_data]
train_output = [data[len(keywords):] for data in training_data]

train_input, test_input, train_output, test_output = train_test_split(train_input, train_output, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_input[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_output[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(np.array(train_input), np.array(train_output), epochs=250, batch_size=5, verbose=1)

Epoch 1/250
50/50 [==============================] - 2s 3ms/step - loss: 3.6503 - accuracy: 0.0280
Epoch 2/250
50/50 [==============================] - 0s 3ms/step - loss: 3.5855 - accuracy: 0.0600
Epoch 3/250
50/50 [==============================] - 0s 3ms/step - loss: 3.4742 - accuracy: 0.1440
Epoch 4/250
50/50 [==============================] - 0s 3ms/step - loss: 3.3432 - accuracy: 0.1760
Epoch 5/250
50/50 [==============================] - 0s 3ms/step - loss: 3.1035 - accuracy: 0.2400
Epoch 6/250
50/50 [==============================] - 0s 3ms/step - loss: 2.8928 - accuracy: 0.2840
Epoch 7/250
50/50 [==============================] - 0s 2ms/step - loss: 2.6455 - accuracy: 0.3640
Epoch 8/250
50/50 [==============================] - 0s 2ms/step - loss: 2.5591 - accuracy: 0.3760
Epoch 9/250
50/50 [==============================] - 0s 2ms/step - loss: 2.3255 - accuracy: 0.3960
Epoch 10/250
50/50 [==============================] - 0s 2ms/step - loss: 2.0822 - accuracy: 0.4520
Epoch 11/

In [ ]:
model.save('chatbot_model.h5', history)

In [ ]:
model = load_model('chatbot_model.h5')

In [ ]:
keywords = pickle.load(open('words.pkl', 'rb'))
classes_list = pickle.load(open('classes.pkl', 'rb'))
intents_data = json.loads(open('intents.json').read())

In [ ]:
def chatbot_response(text):
    tokenized_sentence = nltk.word_tokenize(text)
    preprocessed_sentence = [lemmatizer.lemmatize(word.lower()) for word in tokenized_sentence]
    bag_of_words = [0] * len(keywords)

    for token in preprocessed_sentence:
        for i, word in enumerate(keywords):
            if word == token:
                bag_of_words[i] = 1

    predictions = model.predict(np.array([bag_of_words]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, prediction] for i, prediction in enumerate(predictions) if prediction > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    intent_list = [{"intent": classes_list[result[0]], "probability": str(result[1])} for result in results]
    predicted_intent = intent_list[0]['intent']

    for intent_data in intents_data['intents']:
        if intent_data['tag'] == predicted_intent:
            responses = intent_data['responses']
            response = random.choice(responses)
            return response

    return "Sorry, I'm not sure how to respond to that."

In [41]:
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)


You: Hello
1/1 [==============================] - 0s 82ms/step
Chatbot: Hi there, how can I help?
You: exit
